### Data processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

In [2]:
data = pd.read_json('train.json')

In [3]:
data['band_1'] = data['band_1'].apply(lambda x: np.array(x).reshape(75,75))
data['band_2'] = data['band_2'].apply(lambda x: np.array(x).reshape(75,75))

In [4]:
train = data.sample(frac=0.8)
val = data[~data.isin(train)].dropna()

In [11]:
band_1_tr = np.concatenate([im for im in train['band_1']]).reshape(-1, 75, 75)
band_2_tr = np.concatenate([im for im in train['band_2']]).reshape(-1, 75, 75)
full_img_tr = np.stack([band_1_tr, band_2_tr], axis=3)

In [12]:
band_1_val = np.concatenate([im for im in val['band_1']]).reshape(-1, 75, 75)
band_2_val = np.concatenate([im for im in val['band_2']]).reshape(-1, 75, 75)
full_img_val = np.stack([band_1_val, band_2_val], axis=3)

In [20]:
type(full_img_tr[1][1][1][1])

numpy.float64

In [37]:
y_train = train['is_iceberg'].values
y_train = np.append(y_train.reshape(-1,1), np.zeros([len(y_train),1]),1)
for i in range(len(y_train)):
    if y_train[i,0] == 0:
        y_train[i,1] = 1
#y_train

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       ..., 
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.]])

### keras libraries

In [7]:
#libraries for neural network
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras import initializers, optimizers

Using TensorFlow backend.


In [13]:
cnnkeras = Sequential()

cnnkeras.add(Conv2D(6, (3, 3), activation='relu', input_shape=(75, 75, 2)))
cnnkeras.add(Conv2D(12, (3, 3), activation='relu'))
cnnkeras.add(Conv2D(24, (3, 3), activation='relu'))
cnnkeras.add(Dense(120, kernel_initializer=initializers.random_normal(stddev=0.01), activation = 'relu'))
#cnnkeras.add(Dropout(0.5))
#cnnkeras.add(Dense(10, kernel_initializer=initializers.random_normal(stddev=0.01), activation = 'relu'))
#cnnkeras.add(Dropout(0.5))
cnnkeras.add(Dense(2, kernel_initializer=initializers.random_normal(stddev=0.01), activation = 'softmax'))
#cnnkeras.add(Dense(1, kernel_initializer=initializers.random_normal(stddev=0.01), activation = 'sigmoid'))


In [14]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
'''beta1- The exponential decay rate for the first moment estimates (e.g. 0.9).
   beta2- The exponential decay rate for the second-moment estimates (e.g. 0.999). 
                This value should be set close to 1.0 on problems with a sparse gradient 

there are many types of optimizers other than adam......sgd, RMSprop, Adagrad
sgd = optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
adagrad = optimizers.Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)
'''

cnnkeras.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics=['accuracy'])
#cnnkeras.compile(optimizer = adam, loss = 'binary_crossentropy', metrics=['accuracy'])

In [39]:
cnnkeras.fit(full_img_tr, y_train, epochs=10, batch_size=32) 

ValueError: Error when checking target: expected dense_2 to have 4 dimensions, but got array with shape (1283, 2)

### Test Submission

In [ ]:
test = pd.read_json('test.json')